<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-09 11:30:27
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.40 C
-------------------
Today PnL: 1.54 L (1.11%)
Current PnL: -29.34 L (-18.71%)
CY Booked + Current PnL: -14.60 L (-9.31%)
-------------------
Total profit:  1.38 L
Total loss:  -30.72 L
-------------------
Total Booked + Current PnL: 12.38 L (9.53%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.52%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.73 L (67.59%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 3.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.32,6.77,11.66,19.23,18732.0,10194.0,160654.0,484.83,25.73,56.0,L-LC,12.67,182.0,0.54,1.15,29.49,XY25,ATH,MINING
50,MASFIN,-0.18,-0.84,22.60,21.57,21957.0,-825.0,97155.0,397.04,-14.63,56.0,H-SC,3.41,168.0,-0.04,0.69,41.78,XR,ATH,FINANCE
77,TTKPRESTIG,1.06,-25.28,33.96,0.09,25572.0,-25478.0,75299.0,770.00,74.81,44.0,M-SC,8.46,253.0,-1.00,0.54,3.63,OX40N,NTT,DURABLES
51,MEDANTA,1.37,-3.28,28.28,24.08,34725.0,-4160.0,122790.0,1486.00,-11.10,56.0,X-SC,11.34,91.0,-0.12,0.88,12.47,XY24,NTT,HEALTHCARE
36,ICICIGI,0.30,-1.09,20.66,19.34,40891.0,-2186.0,197923.0,2252.93,-21.92,49.0,X-MC,3.66,71.0,-0.05,1.41,14.67,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,14.23,-19.98,100.92,60.77,147597.0,-36524.0,146251.0,521.00,68.60,65.0,H-SC,3.69,158.0,-0.25,1.04,32.59,XR,NTT,JEWELLERY
7,ATULAUTO,11.86,-14.00,69.26,45.57,117424.0,-27591.0,169541.0,844.00,3672.97,74.0,M-SC,8.60,245.0,-0.23,1.21,28.30,XY24,NTT,AUTO
83,WHIRLPOOL,4.27,-28.58,154.73,81.92,126857.0,-32812.0,81986.0,2270.00,-58.16,65.0,M-SC,32.66,218.0,-0.26,0.59,15.17,XR,NTT,DURABLES
55,RAJESHEXPO,4.21,-65.23,187.78,0.07,89571.0,-89477.0,47700.0,518.00,1648.74,56.0,L-SC,11.09,269.0,-1.00,0.34,18.63,OX40N,NTT,JEWELLERY
71,TANLA,3.77,-43.87,291.41,119.69,442870.0,-118786.0,151975.0,1943.92,-43.33,49.0,H-SC,18.92,150.0,-0.27,1.08,20.06,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,-2.58,-1.76,22.90,20.74,45707.0,-3575.0,199595.0,446.00,28.81,49.0,M-MC,3.72,189.0,-0.08,1.42,9.99,XY25,NTT,FINANCE
65,SIEMENS,-2.24,-16.57,50.45,25.51,78324.0,-30845.0,155250.0,4671.50,-4.77,51.0,H-LC,3.47,50.0,-0.39,1.11,14.41,AR,ATH,ELECTRICAL
52,PAGEIND,-1.97,-11.77,48.04,30.62,100480.0,-27894.0,209160.0,51605.08,-37.34,53.0,X-MC,19.75,57.0,-0.28,1.49,7.20,X40,ATH,APPARELS
13,BERGEPAINT,-0.97,-16.66,45.36,21.14,85939.0,-37880.0,189459.0,680.00,-22.51,29.0,X-MC,9.15,75.0,-0.44,1.35,1.06,XY24,NTT,PAINTS
48,KANSAINER,-0.82,-29.05,59.93,13.47,114665.0,-78336.0,191331.0,340.00,-70.84,32.0,H-SC,6.13,159.0,-0.68,1.37,0.00,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1.65,-0.71,109.41,107.91,160383.0,-1055.0,146589.0,1641.55,-19.34,62.0,M-SC,11.35,220.0,-0.01,1.05,10.28,OX40N,ATH,CABLES
50,MASFIN,-0.18,-0.84,22.60,21.57,21957.0,-825.0,97155.0,397.04,-14.63,56.0,H-SC,3.41,168.0,-0.04,0.69,41.78,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1.65,-0.71,109.41,107.91,160383.0,-1055.0,146589.0,1641.55,-19.34,62.0,M-SC,11.35,220.0,-0.01,1.05,10.28,OX40N,ATH,CABLES
66,SIS,1.16,-20.75,53.02,21.26,46651.0,-23044.0,87988.0,528.00,2080.31,66.0,H-SC,4.53,163.0,-0.49,0.63,21.30,OX40N,NTT,MISC
72,TATAELXSI,0.51,-24.70,75.33,32.03,74784.0,-32560.0,99275.0,9161.00,-18.56,42.0,H-SC,6.33,157.0,-0.44,0.71,11.20,OX40N,NTT,IT
77,TTKPRESTIG,1.06,-25.28,33.96,0.09,25572.0,-25478.0,75299.0,770.00,74.81,44.0,M-SC,8.46,253.0,-1.00,0.54,3.63,OX40N,NTT,DURABLES
40,INDIGOPNTS,2.15,-26.79,36.62,0.02,46798.0,-46765.0,127794.0,1408.00,94.75,35.0,M-SC,5.87,240.0,-1.00,0.91,11.16,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,0.32,6.77,11.66,19.23,18732.0,10194.0,160654.0,484.83,25.73,56.0,L-LC,12.67,182.0,0.54,1.15,29.49,XY25,ATH,MINING
1,ABB,-0.28,6.41,36.79,45.57,102423.0,16781.0,278400.0,7934.00,-31.59,72.0,H-MC,3.84,121.0,0.16,1.99,23.63,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.28,6.41,36.79,45.57,102423.0,16781.0,278400.0,7934.00,-31.59,72.0,H-MC,3.84,121.0,0.16,1.99,23.63,AR,NTT,ELECTRICAL
26,FINCABLES,1.65,-0.71,109.41,107.91,160383.0,-1055.0,146589.0,1641.55,-19.34,62.0,M-SC,11.35,220.0,-0.01,1.05,10.28,OX40N,ATH,CABLES
39,INDIAMART,0.07,-3.74,114.76,106.72,136243.0,-4616.0,118720.0,4810.62,-52.34,52.0,H-SC,8.13,138.0,-0.03,0.85,18.76,AR,ATH,MISC
50,MASFIN,-0.18,-0.84,22.60,21.57,21957.0,-825.0,97155.0,397.04,-14.63,56.0,H-SC,3.41,168.0,-0.04,0.69,41.78,XR,ATH,FINANCE
84,WIPRO,0.16,-5.21,81.74,72.27,131286.0,-8834.0,160614.0,420.00,-18.78,37.0,M-LC,3.55,101.0,-0.07,1.15,3.24,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-0.97,-16.66,45.36,21.14,85939.0,-37880.0,189459.0,680.00,-22.51,29.0,X-MC,9.15,75.0,-0.44,1.35,1.06,XY24,NTT,PAINTS
6,ASIANPAINT,0.70,-15.49,43.12,20.95,91745.0,-39002.0,212766.0,3460.25,-16.37,30.0,X-LC,8.79,36.0,-0.43,1.52,14.62,X40,ATH,PAINTS
42,INFY,-0.11,2.48,31.00,34.24,99402.0,7762.0,320650.0,1972.00,-20.32,31.0,X-LC,5.73,6.0,0.08,2.29,10.97,X40,NTT,IT
8,AWL,-0.65,-31.94,130.96,57.19,290678.0,-104168.0,221959.0,485.00,-67.84,34.0,X-MC,16.82,58.0,-0.36,1.58,1.16,XY24,NTT,FMCG
81,VBL,1.80,-9.72,50.27,35.67,143347.0,-30688.0,285154.0,671.64,-20.46,35.0,X-LC,1.05,5.0,-0.21,2.04,2.79,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,0.29,-18.89,46.16,18.55,141619.0,-71437.0,306800.0,4311.59,-30.10,35.0,X-LC,0.94,3.0,-0.50,2.19,4.38,X40,ATH,IT
81,VBL,1.80,-9.72,50.27,35.67,143347.0,-30688.0,285154.0,671.64,-20.46,35.0,X-LC,1.05,5.0,-0.21,2.04,2.79,X40N,ATH,FMCG
5,ANGELONE,1.67,10.00,12.91,24.20,43347.0,30512.0,335762.0,3033.00,56.91,58.0,X-SC,1.61,99.0,0.70,2.40,40.62,X40N,NTT,FINANCE
21,DABUR,0.43,-0.19,43.98,43.70,108891.0,-476.0,247592.0,735.00,-8.51,51.0,X-MC,1.96,73.0,-0.00,1.77,14.83,XY24,BTT,FMCG
34,HINDUNILVR,-0.48,-6.15,21.12,13.67,50952.0,-15813.0,241250.0,2922.00,-13.11,57.0,X-LC,2.44,9.0,-0.31,1.72,13.89,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-0.97,-16.66,45.36,21.14,85939.0,-37880.0,189459.0,680.00,-22.51,29.0,X-MC,9.15,75.0,-0.44,1.35,1.06,XY24,NTT,PAINTS
8,AWL,-0.65,-31.94,130.96,57.19,290678.0,-104168.0,221959.0,485.00,-67.84,34.0,X-MC,16.82,58.0,-0.36,1.58,1.16,XY24,NTT,FMCG
12,BATAINDIA,2.23,-44.01,141.46,35.20,101919.0,-56622.0,72048.0,2096.00,-5.56,39.0,X-SC,21.30,93.0,-0.56,0.51,2.39,X40,NTT,FOOTWEAR
81,VBL,1.80,-9.72,50.27,35.67,143347.0,-30688.0,285154.0,671.64,-20.46,35.0,X-LC,1.05,5.0,-0.21,2.04,2.79,X40N,ATH,FMCG
2,ABBOTINDIA,0.20,-8.01,28.36,18.07,46658.0,-14335.0,164520.0,35195.00,-21.26,46.0,X-MC,9.49,64.0,-0.31,1.17,2.95,X40,ATH,PHARMA


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,1.51,-28.24,99.06,42.84,46209.0,-18359.0,46647.0,424.00,-53.17,59.0,X-SC,20.81,83.0,-0.40,0.33,12.21,XY24,NTT,MISC
58,RELAXO,0.31,-52.03,217.49,52.31,151449.0,-75525.0,69635.0,1176.00,-48.89,41.0,X-SC,12.83,92.0,-0.50,0.50,3.55,X40N,NTT,FOOTWEAR
12,BATAINDIA,2.23,-44.01,141.46,35.20,101919.0,-56622.0,72048.0,2096.00,-5.56,39.0,X-SC,21.30,93.0,-0.56,0.51,2.39,X40,NTT,FOOTWEAR
51,MEDANTA,1.37,-3.28,28.28,24.08,34725.0,-4160.0,122790.0,1486.00,-11.10,56.0,X-SC,11.34,91.0,-0.12,0.88,12.47,XY24,NTT,HEALTHCARE
35,HONAUT,1.82,-20.10,79.37,43.32,103292.0,-32732.0,130140.0,58357.33,-31.39,46.0,X-SC,9.59,90.0,-0.32,0.93,5.67,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.80,-20.79,39.85,10.78,75860.0,-49953.0,190365.0,452.00,-53.00,44.0,X-LC,18.80,1.0,-0.66,1.36,4.85,X40,NTT,FMCG
75,TMPV,2.31,-22.51,58.71,22.98,102764.0,-50861.0,175037.0,600.00,-19.41,65.0,X-LC,3.99,2.0,-0.49,1.25,11.90,XY24,NTT,AUTO
73,TCS,0.29,-18.89,46.16,18.55,141619.0,-71437.0,306800.0,4311.59,-30.10,35.0,X-LC,0.94,3.0,-0.50,2.19,4.38,X40,ATH,IT
81,VBL,1.80,-9.72,50.27,35.67,143347.0,-30688.0,285154.0,671.64,-20.46,35.0,X-LC,1.05,5.0,-0.21,2.04,2.79,X40N,ATH,FMCG
42,INFY,-0.11,2.48,31.00,34.24,99402.0,7762.0,320650.0,1972.00,-20.32,31.0,X-LC,5.73,6.0,0.08,2.29,10.97,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,2.55,-31.59,94.25,32.89,49782.0,-24388.0,52819.0,1800.00,-752.02,59.0,L-MC,11.48,259.0,-0.49,0.38,45.46,XR,NTT,BANKS
67,SONACOMS,3.47,-8.09,51.33,39.09,47725.0,-8182.0,92977.0,804.02,-28.09,67.0,M-SC,1.74,227.0,-0.17,0.66,31.53,AR,ATH,AUTO
50,MASFIN,-0.18,-0.84,22.60,21.57,21957.0,-825.0,97155.0,397.04,-14.63,56.0,H-SC,3.41,168.0,-0.04,0.69,41.78,XR,ATH,FINANCE
14,BSOFT,1.77,-19.61,87.57,50.79,99013.0,-27586.0,113067.0,831.70,3.21,56.0,H-SC,7.13,171.0,-0.28,0.81,31.65,XR,ATH,IT
79,VAIBHAVGBL,14.23,-19.98,100.92,60.77,147597.0,-36524.0,146251.0,521.00,68.60,65.0,H-SC,3.69,158.0,-0.25,1.04,32.59,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,14.23,-19.98,100.92,60.77,147597.0,-36524.0,146251.0,521.00,68.60,65.0,H-SC,3.69,158.0,-0.25,1.04,32.59,XR,NTT,JEWELLERY
5,ANGELONE,1.67,10.00,12.91,24.20,43347.0,30512.0,335762.0,3033.00,56.91,58.0,X-SC,1.61,99.0,0.70,2.40,40.62,X40N,NTT,FINANCE
67,SONACOMS,3.47,-8.09,51.33,39.09,47725.0,-8182.0,92977.0,804.02,-28.09,67.0,M-SC,1.74,227.0,-0.17,0.66,31.53,AR,ATH,AUTO
7,ATULAUTO,11.86,-14.00,69.26,45.57,117424.0,-27591.0,169541.0,844.00,3672.97,74.0,M-SC,8.60,245.0,-0.23,1.21,28.30,XY24,NTT,AUTO
1,ABB,-0.28,6.41,36.79,45.57,102423.0,16781.0,278400.0,7934.00,-31.59,72.0,H-MC,3.84,121.0,0.16,1.99,23.63,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.25
1,25,44.52
2,50,76.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.70
MC    30.75
LC    23.60
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.63
X40      23.30
X40N     15.12
AR        8.95
XR        8.94
XY25      8.77
OX40N     7.44
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.09
H-SC    22.97
X-LC    20.19
M-SC    11.59
X-SC    10.37
H-MC     4.86
M-MC     1.42
L-LC     1.15
M-LC     1.15
H-LC     1.11
L-SC     0.77
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.92,-13.30,51.38
FINANCE,13.44,-10.85,54.47
IT,12.24,-24.62,86.88
MISC,7.12,-30.31,83.61
ELECTRICAL,6.08,-10.36,50.62
PAINTS,5.15,-28.00,47.02
INSURANCE,4.72,-2.49,37.56
PHARMA,4.02,-4.35,37.20
AUTO,3.12,-19.80,61.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3241169.0,21
AR,1318511.0,10
XR,1241890.0,12
X40,1231250.0,15
X40N,1004602.0,10
OX40N,750534.0,10
XY25,391201.0,6
SR,293876.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3576513.0,24
X-MC,1650481.0,16
M-SC,1437308.0,15
X-LC,1063757.0,12
X-SC,839222.0,9
H-MC,398541.0,3
L-SC,183380.0,2
M-LC,131286.0,1
H-LC,78324.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1233283.0      6
           AR         931715.0      5
M-SC       XY24       806655.0      6
H-SC       XR         791301.0      7
X-MC       X40        547772.0      7
X-LC       X40        514741.0      6
X-MC       XY24       485508.0      3
           X40N       404478.0      4
X-SC       X40N       368263.0      4
M-SC       OX40N      334625.0      5
H-SC       OX40N      326338.0      4
X-SC       XY24       302222.0      3
H-SC       SR         293876.0      2
X-LC       X40N       231861.0      2
X-MC       XY25       212723.0      2
H-MC       XY24       210385.0      1
X-LC       XY24       203116.0      2
H-MC       AR         188156.0      2
M-SC       XR         175712.0      2
X-SC       X40        168737.0      2
M-LC       XR         131286.0      1
M-SC       AR         120316.0      2
X-LC       XY25       114039.0      2
L-SC       XR          93809.0      1
           OX40N       89571.0      1
H-LC       AR          78324.0      1
L-MC       XR          49782.0      1
M-MC       XY25        45707.0      1
L-LC       XY25        18732.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
